# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier



---
## Step 0: Load The Data

In [ ]:
### Load pickled data and size check

import pickle
import numpy as np

#Fill this in based on where you saved the training and testing data
training_file = 'traffic_signs_data/train.p'
validation_file= 'traffic_signs_data/valid.p'
testing_file = 'traffic_signs_data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train_src, y_train = train['features'], train['labels']
X_valid_src, y_valid = valid['features'], valid['labels']
X_test_src, y_test = test['features'], test['labels']

#Verification
assert(len(X_train_src) == len(y_train))
assert(len(X_valid_src) == len(y_valid))
assert(len(X_test_src) == len(y_test))

---

## Step 1: Dataset Summary & Exploration


### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Init values of dataset sizes

#Number of training examples
n_train = len(X_train_src)

#Number of validation examples
n_validation = len(X_valid_src)

#Number of testing examples.
n_test = len(X_test_src)

#What's the shape of an traffic sign image?
image_shape = X_train_src[0].shape

#How many unique classes/labels there are in the dataset.
n_classes = len(set(y_valid))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:
# Creating dictionary ClassId -> NameStr
import csv

signnames_file = 'signnames.csv'
signs_dict = {}
with open(signnames_file) as signnames:
    reader = csv.DictReader(signnames, delimiter=',')
    for row in reader:
        signs_dict[int(row['ClassId'])] = row['SignName']

In [ ]:
### Displaying some images from validation dataset
import random

demo_pic_cnt = 3

plt.figure(figsize=(15,10))
for i in range(demo_pic_cnt):
    index = random.randint(0, len(X_valid_src))
    demo_img_src = X_valid_src[index]
    title_str_src = signs_dict[y_valid[index]]

    plt.subplot(1, demo_pic_cnt, i+1)
    plt.imshow(demo_img_src)
    plt.title(title_str_src)

In [ ]:
### Counting and plotting number of images per class id


id_cnt = np.array([[idx, y_train.tolist().count(idx)] for idx in set(y_train.tolist())])

plt.figure(figsize=(20,10))
plt.stem(id_cnt[:,0], id_cnt[:,1])
plt.xlabel('Class ID');
plt.ylabel('Number of elements in class')

----

## Step 2: Design and Test a Model Architecture



### Pre-process the Data Set (normalization, grayscale, etc.)

In [ ]:
### Data preprocessing
import numpy as np

def normalize_data(X):
    X_res = np.zeros(X.shape, dtype=float)
    for i in range(len(X)):
        img = X[i].astype(dtype=float)
        img -= np.min(img)
        img /= np.max(img)
        X_res[i] = img
    return X_res
        
X_train = normalize_data(X_train_src)
X_valid = normalize_data(X_valid_src)
X_test = normalize_data(X_test_src)

In [ ]:
# Displaying image preprocessing and normalisation
import random

demo_pic_cnt = 3

plt.figure(figsize=(15,10))
for i in range(demo_pic_cnt):
    index = random.randint(0, len(X_valid))
    demo_img_src = X_valid_src[index]
    demo_img = X_valid[index]
    
    title_str_src = signs_dict[y_valid[index]] + ', before'
    title_str = signs_dict[y_valid[index]] + ', after'

    plt.subplot(2, demo_pic_cnt, i+1)
    plt.imshow(demo_img_src)
    plt.title(title_str_src)
    
    plt.subplot(2, demo_pic_cnt, i+1+demo_pic_cnt)
    plt.imshow(demo_img)
    plt.title(title_str)

### Model Architecture

In [ ]:
### LeNet architecture with 32*32*3 input size

from tensorflow.contrib.layers import flatten


def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
    x = tf.nn.bias_add(x, b)
    return x


def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='VALID')


def LeNet(x, keep_probability):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    n_classes = 43
    
    weights = {
        'wc1': tf.Variable(tf.truncated_normal([5, 5, 3, 6], mean=mu, stddev=sigma)),
        'wc2': tf.Variable(tf.truncated_normal([5, 5, 6, 16], mean=mu, stddev=sigma)),
        'wfc1':tf.Variable(tf.truncated_normal([400, 120], mean=mu, stddev=sigma)),
        'wfc2':tf.Variable(tf.truncated_normal([120, 84], mean=mu, stddev=sigma)),
        'out': tf.Variable(tf.truncated_normal([84, n_classes], mean=mu, stddev=sigma))}

    biases = {
        'bc1': tf.Variable(tf.truncated_normal([6], mean=mu, stddev=sigma)),
        'bc2': tf.Variable(tf.truncated_normal([16], mean=mu, stddev=sigma)),
        'bfc1':tf.Variable(tf.truncated_normal([120], mean=mu, stddev=sigma)),
        'bfc2':tf.Variable(tf.truncated_normal([84], mean=mu, stddev=sigma)),
        'out': tf.Variable(tf.truncated_normal([n_classes], mean=mu, stddev=sigma))}
    
    #Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], 1)
    
    #Activation.
    conv1 = tf.nn.relu(conv1)
    
    #Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1p = maxpool2d(conv1, 2)
    
    #Layer 2: Convolutional. Output = 10x10x16.
    conv2 = conv2d(conv1p, weights['wc2'], biases['bc2'], 1)
    
    #Activation.
    conv2 = tf.nn.relu(conv2)
    
    #Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2p = maxpool2d(conv2, 2)                   
    
    #Flatten. Input = 5x5x16. Output = 400.
    conv2p = tf.contrib.layers.flatten(conv2p)                  
        
    #Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1 = tf.add(tf.matmul(conv2p, weights['wfc1']), biases['bfc1'])

    #Activation.
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, keep_probability)

    #Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2 = tf.add(tf.matmul(fc1, weights['wfc2']), biases['bfc2'])
    
    #Activation.
    fc2 = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, keep_probability)
    
    #Layer 5: Fully Connected. Input = 84. Output = 43.
    logits = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    
    return logits, conv1

In [ ]:
### Network for testing purposes (not used in trainig, just for debugging purpose)
def TestNet(x, keep_probability):  
    
    mu = 0
    sigma = 0.1
    n_classes = 43
    weights = {'wc1': tf.Variable(tf.truncated_normal([5, 5, 1, 6], mean=mu, stddev=sigma)),
               'wfc1out':tf.Variable(tf.truncated_normal([4704, n_classes], mean=mu, stddev=sigma))}
    biases = {'bc1': tf.Variable(tf.truncated_normal([6], mean=mu, stddev=sigma)), 
              'bfc1out':tf.Variable(tf.truncated_normal([n_classes], mean=mu, stddev=sigma)),}
    
    #Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], 1)
    
    #Flatten. Input = 28x28x6. Output = 4704.
    conv2 = tf.contrib.layers.flatten(conv1)
    
    logits = tf.add(tf.matmul(conv2, weights['wfc1out']), biases['bfc1out'])
    
    return logits

### Train, Validate and Test the Model

In [ ]:
### Setting down network parameters

import tensorflow as tf

EPOCHS = 100
BATCH_SIZE = 512
rate = 0.001

In [ ]:
### Initial declaration

#batch place holders
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))

#Dropout variable
keep_probability = tf.placeholder(tf.float32)

one_hot_y = tf.one_hot(y, n_classes)

In [ ]:
### Network pipeline

# layer_out will be used to show weight activation on a custom data input
[logits, layer_out] = LeNet(x, keep_probability)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
### Model evaluation

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    dropout = 1.0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_probability: dropout})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
### Model train

from sklearn.utils import shuffle

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        dropout = 0.4
#         dropout = 0.5
#         dropout = 0.7
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_probability: dropout})
            
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet_traffic')
    print("Model saved")

In [ ]:
### Evaluation of model on test dataset

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    train_accuracy = evaluate(X_train, y_train)
    print("Train Accuracy = {:.3f}".format(train_accuracy))

    valid_accuracy = evaluate(X_valid, y_valid)
    print("Validation Accuracy = {:.3f}".format(valid_accuracy))
    
    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

---

## Step 3: Test a Model on New Images


### Load and Output the Images

In [ ]:
### Reading custom data from internet to variable

import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt

path = 'my_data/cropped/sized/*.bmp'
files = glob.glob(path)

batch_size = len(files)
batch_x = np.zeros((batch_size, 32, 32, 3), dtype=np.uint8)

for i in range(batch_size):
    img = cv2.imread(files[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    batch_x[i, :, :, :] = np.copy(img)

X_mydata = normalize_data(batch_x)

In [ ]:
### Reading the csv-file with class id number and its name pairs

import csv

# Create Dictionary
signnames_file = 'signnames.csv'
signs_dict = {}
with open(signnames_file) as signnames:
    reader = csv.DictReader(signnames, delimiter=',')
    for row in reader:
        signs_dict[int(row['ClassId'])] = row['SignName']

### Predict the Sign Type for Each Image

In [ ]:
### Processing custom data and visualizating results

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    
    prediction = tf.argmax(tf.nn.softmax(logits), 1)
    top_k      = tf.nn.top_k(tf.nn.softmax(logits), k=5)
    dropout = 1
    results, probabilities = sess.run([prediction, top_k], feed_dict={x: X_mydata, keep_probability: dropout})


columns = 5
plt.figure(figsize=(20,10))
for i in range(len(X_mydata)):
    plt.subplot(len(X_mydata) / columns + 1, columns, i + 1)
    plt.imshow(X_mydata[i])
    plt.title(signs_dict[results[i]])


### Analyze Performance

In [ ]:
### Performance calculation

total_num = len(X_mydata)
correct_num = 8
mydata_accuracy = correct_num / total_num

print("Accuracy for custom data from internet is %.3f" % mydata_accuracy)

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

In [ ]:
### Printing pseudoprobabilities and classes

np.set_printoptions(precision = 8, suppress = True)

print(probabilities.values)
print(probabilities.indices)

top_k_num = 5

for i in range(len(X_mydata)):
#     print("| Probability         	|     Prediction	        					|")
#     print("|:---------------------:|:---------------------------------------------:|")
    print("Pic")
    for j in range(top_k_num):
        print("|%0.3f | %s" % (probabilities.values[i,j], signs_dict[probabilities.indices[i,j]]))

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images



In [ ]:
### Visualization of a two first layers bypassing by the custom data
def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    activation = sess.run(tf_activation, feed_dict={x: image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    
    for i in range(len(X_mydata)):
        test_input = np.expand_dims(X_mydata[i],0)
        outputFeatureMap(test_input, layer_out, plt_num=i+1)